## Load Libraries and Packages

In [1]:
import datetime
import numpy as np
from numpy import double
import pandas as pd

## Load Data

In [2]:
print("Loading data start...")
train = pd.read_csv("input/train.csv")
test = pd.read_csv("input/test.csv")
store = pd.read_csv("input/store.csv")
states = pd.read_csv("input/store_states.csv")
print("Complete!   : )")

Loading data start...
Complete!   : )


/Users/jfdarre/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


## Process Data:
#### 1: Impute open = 1 for missing open test dasta
#### 2: Separate date column into year, month, day
#### 3: Convert date column to type 'date' and extract:
* day_of_year
* week_of_year
* quarter
* month start
* month end
* quarter start
* quarter end

#### 4: Drop date and standardize StateHoliday
#### 5: Create promotion one feature
#### 6: Merge data
#### 7: Dumify categorical variables

In [3]:
# 1: Impute Open = 1 for store 622 in test data
test['Open'].fillna(1, inplace=True)
print 'Step 1 Complete.'

Step 1 Complete.


In [4]:
# 2: Separate date columns
test['year'] = test.Date.apply(lambda x: x.split('-')[0])
test['year'] = test['year'].astype(int)
test['month'] = test.Date.apply(lambda x: x.split('-')[1])
test['month'] = test['month'].astype(int)
test['day'] = test.Date.apply(lambda x: x.split('-')[2])
test['day'] = test['day'].astype(int)
print 'Step 2 Complete.'

Step 2 Complete.


In [5]:
# 3: Withdraw multiple date features
test['Date'] = pd.to_datetime(test['Date'])
test['day_of_year'] = test['Date'].dt.dayofyear
test['week_of_year'] = test['Date'].dt.weekofyear
test['quarter'] = test['Date'].dt.quarter
test['is_month_start'] = test['Date'].dt.is_month_start.astype(int)
test['is_month_end'] = test['Date'].dt.is_month_end.astype(int)
test['is_quarter_start'] = test['Date'].dt.is_quarter_start.astype(int)
test['is_quarter_end'] = test['Date'].dt.is_quarter_end.astype(int)
print 'Step 3 Complete.'

Step 3 Complete.


In [6]:
# 4: Drop date and standardize StateHoliday column.
# train = train.drop('Date',1)
test.ix[(test['StateHoliday'] == 0), 'StateHoliday'] = '0'
#train['StateHoliday'] = map(lambda x: '0' if x == 0 else x, train['StateHoliday'])
print 'Step 4 Complete.'

Step 4 Complete.


In [7]:
# 5: Create duration of promo 1
test = test.sort(columns = ['Store', 'Date'], ascending = True, na_position = 'last')
test['PromoFirstDate'] = 0
test.ix[(train['Promo'] == 1 ) & (test['Promo'].shift(1) ==0), 'PromoFirstDate'] = 1
print 'Step 5 Complete.'

Step 5 Complete.


In [8]:
# 6: Merge store and location features with training data
tm_temp = pd.merge(test, store, on = 'Store')
tm = pd.merge(tm_temp, states, on = 'Store')
print 'Step 6 Complete.'


Step 6 Complete.


# Output CSV files for R

In [9]:
tm.to_csv("data/test_store_states_R.csv")

In [10]:
# 7: Create features for all state holidays, store types and assortments
tm_SH    = pd.get_dummies(tm.StateHoliday,'StateHoliday')
tm_ST    = pd.get_dummies(tm.StoreType, 'StoreType')
tm_AS    = pd.get_dummies(tm.Assortment, 'Assortment')
tm_State = pd.get_dummies(tm.State, 'State')

tm_dummy = pd.concat([tm, tm_State[tm_State.columns[1:]]],axis = 1)
tm_dummy = pd.concat([tm_dummy, tm_SH[tm_SH.columns[1:]]],axis = 1)
tm_dummy = pd.concat([tm_dummy, tm_ST[tm_ST.columns[1:]]],axis = 1)
tm_final = pd.concat([tm_dummy, tm_AS[tm_AS.columns[1:]]],axis = 1)
tm_final.drop(['StateHoliday', 'StoreType', 'Assortment', 'State'],1,inplace=True)
print 'Step 7 Complete.'

tm_final.head()

Step 7 Complete.


,Id,Store,DayOfWeek,Date,Open,Promo,SchoolHoliday,year,month,day,...,State_HH,State_NW,State_RP,State_SH,StateHoliday_a,StoreType_b,StoreType_c,StoreType_d,Assortment_b,Assortment_c
0,40233,1,6,2015-08-01,1,0,1,2015,8,1,...,0,0,0,0,0,0,1,0,0,0
1,39377,1,7,2015-08-02,0,0,1,2015,8,2,...,0,0,0,0,0,0,1,0,0,0
2,38521,1,1,2015-08-03,1,1,1,2015,8,3,...,0,0,0,0,0,0,1,0,0,0
3,37665,1,2,2015-08-04,1,1,1,2015,8,4,...,0,0,0,0,0,0,1,0,0,0
4,36809,1,3,2015-08-05,1,1,1,2015,8,5,...,0,0,0,0,0,0,1,0,0,0


In [12]:
tm_final.to_csv("data/test_store_states_Python.csv")